# Environment:

Login into the GDAV server

`$ ssh youruser@IP`

Create a directory in your home folder called `compgenomics_ex1`

`$ cd javier.fgonzalez`

`$ mkdir compgenomics_ex1`

and Enter the directory 

`$ cd compgenomics_ex1`

All the files needed for this exercise are copied in the GDAV server at `/home/compgenomics/4proteomes/`. Make sure you can see them and take a few seconds to understand what they contain: 

```
$ ls /home/compgenomics/4proteomes/
4proteomes.faa    -> all protein sequences from 4 species: human, elefant, Zebrafish and Ciona intestinalis. 
G3T0S8_LOXAF.faa  -> protein sequence of the elefant gene called G3T0G8
TPH1A_rerio.faa   ->  protein sequence of the Zebrafish gene called TPH1A
TPH2_human.faa    -> protein sequence of Human TPH2 
scripts/          ->  a directory with ad hoc programs and scripts`
```

Puedes hacer esto para ver cómo están hechos los fasta: 

`$ cd ~`

`$ head /home/compgenomics/4proteomes/G3T0S8_LOXAF.faa` Con esto puedes ver que las entradas para elefante contienen _LOXAF en el encabezado

Las de pez cebra tienen _DANRE

Las de humano contienen _HUMAN

Probando con greps encontré las de Ciona intestinalis

`$ grep 'CIO' /home/compgenomics/4proteomes/4proteomes.faa`

Con esto encontré que las de Ciona contienen _CIOIN


## Tools needed
(already installed in the GDAV server)

- BLAST+ 

# Exercise

## Goal 1 (Loxodonta orthologs)

Antes de nada vuelve a hacer:

` $ cd compgenomics_ex1`

Using just BLAST reciprocal searches, could you identify which is the human ortholog of the Loxodonta protein G3T0S8? (remember that the protein sequence of G3T0S8 is available in `/home/compgenomics/4proteomes/G3T0S8_LOXAF.faa`)

### Protocol

#### 1. Create a BLAST database
Make a blast database containing the 4 input proteomes (Human, Danio rerio, Ciona intestinalis, and Loxodonta africana) and name the database as “4proteomes.blastdb’. All proteomes are already merged into a single FASTA file `/home/compgenomics/4proteomes/4proteomes.faa`

```
$ makeblastdb -dbtype prot -in /home/compgenomics/4proteomes/4proteomes.faa -out 4proteomes.blastdb
```

You should now see something like this in your exercise home folder: 
```
(base) [test@gdav compgenomics_ex1]$ ls -l
total 48604
-rw-rw-r--. 1 test test  7434160 oct 15 15:58 4proteomes.blastdb.phr
-rw-rw-r--. 1 test test   683920 oct 15 15:58 4proteomes.blastdb.pin
-rw-rw-r--. 1 test test 41649539 oct 15 15:58 4proteomes.blastdb.psq
```
#### 2. Find G3T0S8_LOXAF.faa homologs. 

Use the `blastp` command to search for all homologs of the G3T0S8 sequence. Use an evalue threshold of 0.001

```
$ blastp -task blastp -query /home/compgenomics/4proteomes/G3T0S8_LOXAF.faa -db 4proteomes.blastdb -outfmt 6 -evalue 0.001

```
Es más cómodo guardarlo en un archivo 

```
$ blastp -task blastp -query /home/compgenomics/4proteomes/G3T0S8_LOXAF.faa -db 4proteomes.blastdb -outfmt 6 -evalue 0.001 >> G3T0S8_homologs
```


#### 3. Answer these questions
1. How many homologs of G3T0S8 are in HUMAN?


```

$ grep '_HUMAN' G3T0S8_homologs | wc -l

```

Te da 4.

2. Which is the closest one?

He hecho:

```
$ grep '_HUMAN' G3T0S8_homologs |sort -g -k 11

```
-k 11 significa que ordenes según la undécima columna (se empieza a contar por 1)

-g significa que hay números exponenciales y que tiene que leerlos bien.


El menor e-value es TPH1_HUMAN_TPH1. Ese es el resultado


3. Are they orthologs?

Hay que hacer blast usando TPH1_HUMAN_TPH1 como query y ver si el mejor match es G3T0S8 (método Best Reciprocal Hits, BRH). En caso afirmativo son ortólogos. En caso negativo puede que sean ortólogos o puede que no (ver apuntes).

Lo primero es guardar el gen TPH1 el solo en un fasta:

```
$ grep -A 1 'TPH1_HUMAN_TPH1' /home/compgenomics/4proteomes/4proteomes.faa >> TPH1_HUMAN.faa
```

-A 1 lo que hace es meter en el output la línea que va después de la línea en la que encontraste el match (ej si quisieras 3 líneas pones -A 3). Solo queremos 1 porque el match está en el encabezado y toda la secuencia cuenta como una única línea.



```
$ blastp -task blastp -query TPH1_HUMAN.faa -db 4proteomes.blastdb -outfmt 6 -evalue 0.001 >> TPH1_homologs
```

```
$ grep '_LOXAF' TPH1_homologs |sort -g -k 11

```

el menor e-value (0) es para el gen G3SPB9_LOXAF_TPH2. Como no ha sido G3T0S8 eso quiere decir que o bien TPH1_HUMAN y G3T0S8 no son ortólogos o bien hay una relación de ortología complicada (ej one to many; many to many, etc.).

#### 4. Save homologous sequences

Vaya esto ya lo hice antes. Estos son otros comandos para hacerlo. No voy a repetirlo.

Extract the sequence of the closest homolog of G3T0S8_LOXAF in HUMAN and save it a new file called `G3T0S8_best_human_hit.faa`. 

There are many different ways to do this. You can open the FASTA file containing all proteomes, search by the sequence name and extract it manually, or you can (as you should) do it from the command line. 

```
$ grep [HUMAN_homolog_in_blast_result] all_species.parsed.fasta -A1 > G3T0S8_best_human_hit.faa
```

#### 5. Now, BLAST G3T0S8_best_human_hit.faa against the same databases. 
```
$ blastp -task blastp -query [HUMAN_seq_file] -db 4proteomes.blastdb -outfmt 6 -evalue 0.001
```

#### 6. Answer these questions
1. Are they reciprocal hits?

No. G3T0S8 aparece en tercer lugar con e-value = 1.89e-155	


2. Are they orthologous with each other?

O bien TPH1_HUMAN y G3T0S8 no son ortólogos o bien hay una relación de ortología complicada (ej one to many; many to many, etc.).

## Goal 2 (Zebrafish orthologs)
Repeat the previous protocol with the Zebrafish sequence found in the file `TPH1A_rerio.faa` (Danio rerio homolog)
###  Answer these questions: 

1. What are the Zebrafish homologs in human? 
2. Are the same as in the Loxodonta example?
3. What is the ortholog in human (based on reciprocal blast)?
4. What could you tell about the gene TPH1B in Danio rerio?


